[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ciri/iese-dsfb/blob/main/notebooks/210-Amazon-Part-II.ipynb)

# Tabular: Amazon Part II

NOTE: The content in this notebook is best rendered in a jupyter notebook or google collab.

Last time, we identified a decline in revenue and overdependence on one category. Today, we'll investigate what might be hurting revenue (like cancellations), and then look at where and to whom Amazon could sell more — ending with a focused growth strategy.

In [1]:
import pandas as pd
df = pd.read_csv('../resources/tabular/Amazon-cleaned.csv')

## Cancellations & Returns

Not all orders are good news. In e-commerce, cancellations and returns create serious operational and financial issues:
* Lost revenue: These orders don’t convert into sales.
* Shipping and handling costs: Often non-refundable.
* Reverse logistics: Costly and time-consuming.
* Chargebacks: Especially if customers dispute transactions.
* Fraud & abuse: Repeated returns or B2C-to-B2B loopholes.
* Money laundering flags: In some cases, abnormal return behavior is a red flag.

Long story short: cancellations and returns hurt revenue.

Let’s measure the size of the problem:

* How many orders were cancelled?
* How many were returned?
* What percentage of total orders does this represent?

So far, you’ve used `.groupby()` for totals and averages. Now let’s introduce `.value_counts()` which is a quick way to count categories in one column, like a faster `.groupby('status').shape`.

In [2]:
# Step 1: Count all status values
# ---> same as: df.groupby('status').size().sort_values(ascending=False)
df['status'].value_counts() 

status
Shipped                          75940
Shipped - Delivered to Buyer     28029
Cancelled                        10756
Shipped - Returned to Seller      1949
Shipped - Picked Up                945
Pending                            639
Pending - Waiting for Pick Up      272
Shipped - Returning to Seller      143
Shipped - Out for Delivery          35
Shipped - Rejected by Buyer         11
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: count, dtype: int64

In [3]:
# Step 2: Filter for problematic orders (cancelled or returned)
cancelled = df[df['status'] == 'Cancelled']
returned  = df[df['status'] == 'Shipped - Returned to Seller']

In [4]:
# Step 3: Calculate totals and percentages
total_orders    = df.shape[0]
cancelled_pct   = cancelled.shape[0] / total_orders * 100
returned_pct    = returned.shape[0] / total_orders * 100
problematic_pct = (cancelled.shape[0]  + returned.shape[0]) / total_orders * 100

print(f"Cancelled Orders: {len(cancelled)} ({cancelled_pct:.2f}%)")
print(f"Returned Orders: {len(returned)} ({returned_pct:.2f}%)")
print(f"Total Problematic Orders: {problematic_pct:.2f}%")

Cancelled Orders: 10756 (9.06%)
Returned Orders: 1949 (1.64%)
Total Problematic Orders: 10.70%


> Observations: High cancellation rates (churn?), high return rates (issues with quality or fit? need to figure this out to avoid expensive logistics).

### Revenue by Customer Type (B2B vs. Regular)

Amazon doesn’t just sell to everyday consumers — it also serves **business buyers** through **Amazon Business**.

| **Customer Type** | **Examples**                                      | **What They Value**                                |
|--------------------|---------------------------------------------------|----------------------------------------------------|
| Regular (B2C)       | Individual shoppers buying for personal use      | Price, fast shipping, product variety              |
| Business (B2B)      | SMEs, schools, retailers buying in bulk          | Discounts, tax invoices, delivery reliability, APIs|

Business customers usually spend more per order — but place fewer orders.  
Regular customers are more frequent, but each transaction is smaller.

Let’s compare:
1. Total revenue by customer type  
2. Average order value (AOV) by customer type  
3. Number of orders by customer type

In [5]:
# Step 1: Group by customer type and summarize metrics
customer_summary = df.groupby('B2B').agg(
    total_revenue   = ('amount_eu', 'sum'),
    avg_order_value = ('amount_eu', 'mean'),
    order_count     = ('order_id', 'count')
).reset_index()
customer_summary

,B2B,total_revenue,avg_order_value,order_count
0,False,856880.31,7.268967,117882
1,True,6503.49,7.714698,843


In [6]:
# Step 2: Convert True/False to readable labels
customer_summary['B2B'] = customer_summary['B2B'].replace({True: 'B2B', False: 'B2C'})
customer_summary

,B2B,total_revenue,avg_order_value,order_count
0,B2C,919207.39,7.797691,117882
1,B2B,6976.65,8.275979,843


**Observations**:

| **Metric**         | **What It Shows**                                                                                |
|--------------------|--------------------------------------------------------------------------------------------------|
| **Order Count** | Regular customers dominate volume — this is the bulk of the business                            |
| **AOV**         | Business customers spend **~8% more per order** (€7.71 vs €7.13) but much less than expected.                                  |
| **Total Revenue** | Business is currently a *tiny slice* of total revenue (~0.75%) — but with higher ticket size |

Potential actionable insight: one last attempt at invigorating and attracting better B2B through e.g. exclusive deals, bulk pricing, repeate order flows? If it doesn't work we may consider spinning off or integrating it into B2C.

In [7]:
#df.ship_service_level.unique()
df.columns

Index(['order_id', 'date', 'status', 'fulfilment', 'sales_channel',
       'ship_service_level', 'style', 'base-SKU', 'SKU', 'category', 'size',
       'ASIN', 'courier_status', 'qty', 'currency', 'amount', 'ship_city',
       'ship_state', 'ship_postal_code', 'ship_country', 'promotion_ids',
       'B2B', 'fulfilled_by', 'amount_eu'],
      dtype='object')

## Most Popular Products by State

Let's say that you’re working as a data scientist or BI developer. The goal is not just to analyze but rather to communicate insights clearly as we've been trying to do before. Sometimes a plot is enough, other times a dynamic dashboard is needed.

Dashboards allow business users (e.g., regional managers) to quickly:

* See how their region is doing
* Spot product trends
* Act on poor performance
* ..

We'll build a simple command-line dashboard first, and then make it interactive.

In [6]:
df.ship_state.unique()

array(['MAHARASHTRA', 'KARNATAKA', 'PUDUCHERRY', 'TAMIL NADU',
       'UTTAR PRADESH', 'CHANDIGARH', 'TELANGANA', 'ANDHRA PRADESH',
       'RAJASTHAN', 'DELHI', 'HARYANA', 'ASSAM', 'JHARKHAND',
       'CHHATTISGARH', 'ODISHA', 'KERALA', 'MADHYA PRADESH',
       'WEST BENGAL', 'NAGALAND', 'GUJARAT', 'UTTARAKHAND', 'BIHAR',
       'JAMMU & KASHMIR', 'PUNJAB', 'HIMACHAL PRADESH',
       'ARUNACHAL PRADESH', 'MANIPUR', 'GOA', 'MEGHALAYA', 'TRIPURA',
       'LADAKH', 'DADRA AND NAGAR', 'SIKKIM', nan, 'ANDAMAN & NICOBAR',
       'RAJSHTHAN', 'NL', 'MIZORAM', 'NEW DELHI',
       'PUNJAB/MOHALI/ZIRAKPUR', 'RJ', 'ORISSA', 'LAKSHADWEEP', 'PB',
       'AR', 'PONDICHERRY', 'RAJSTHAN'], dtype=object)

Let's focus on one state to understand its sales profile. We'll start with Karnataka, one of the largest markets

In [7]:
# Filter data for Karnataka
karnataka = df[df['ship_state'] == 'KARNATAKA']

In [8]:
# Calculate the number of orders placed in this state
total_orders = len(karnataka)

# Calculate total revenue generated
total_revenue = karnataka['amount_eu'].sum()

# Average revenue per order — helps identify spending behavior
avg_order_value = karnataka['amount_eu'].mean()

# Find the most popular product category by quantity sold
top_category = (
    karnataka.groupby('category')['qty']
    .sum()
    .sort_values(ascending=False)
    .head(1)
)

# Print the summary stats
print(f"Total Orders: {total_orders}")
print(f"Total Revenue: €{total_revenue:,.2f}")
print(f"Average Order Value: €{avg_order_value:,.2f}")
print(f"Top Category: {top_category.index[0]} ({top_category.values[0]} units)")

Total Orders: 16050
Total Revenue: €115,144.39
Average Order Value: €7.17
Top Category: kurta (6085 units)


Now, we wrap whatever you came up with in a function

In [12]:
def draw_state_dashboard(state):
    # Standardize input casing
    subset = df[df['ship_state'] == state.upper()]
    
    if subset.empty:
        print("No data for selected state.")
        return

    # Core KPIs
    total_orders = len(subset)
    total_revenue = subset['amount_eu'].sum()
    avg_order_value = subset['amount_eu'].mean()
    top_category = (
        subset.groupby('category')['qty']
        .sum()
        .sort_values(ascending=False)
        .head(1)
    )

    # Print the summary
    print(f"--- Dashboard for {state.title()} ---")
    print(f"Total Orders: {total_orders}")
    print(f"Total Revenue: €{total_revenue:,.2f}")
    print(f"Average Order Value: €{avg_order_value:,.2f}")
    print(f"Top Category: {top_category.index[0]} ({top_category.values[0]} units)")

draw_state_dashboard("karnataka")

--- Dashboard for Karnataka ---
Total Orders: 16050
Total Revenue: €123,519.42
Average Order Value: €7.70
Top Category: kurta (6085 units)


**Dashboard**

Wonderful, all that is needed at this point is to make it interactive with a dropdown. Note: this code assumes that you are using a notebook like google collab or jupyter lab. Notably it won't work in VS Code.

Note: You will need to install and enable widgets, it doesn't always work on every environment.

In [13]:
! pip install ipywidgets==8.0.2
! pip install jupyterlab_widgets==3.0.13

In [14]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Dropdown with all cleaned states
output = widgets.Output()
state_dropdown = widgets.Dropdown(
    options=sorted(df['ship_state'].dropna().unique()),
    description='Select a State:',
    style={'description_width': 'initial'}
)

# When a user selects a different state, redraw the dashboard
def on_state_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
       with output:
            clear_output(wait=True)
            draw_state_dashboard(change['new'])

# Connect the dropdown to the callback
state_dropdown.observe(on_state_change)
display(state_dropdown, output)

Dropdown(description='Select a State:', options=('ANDAMAN & NICOBAR', 'ANDHRA PRADESH', 'AR', 'ARUNACHAL PRADE…

Output()

### BONUS: add some analysis from these and the previous classes to make it prettier

Note: in production you will want to use plotly dash, which functions similar to what we've seen above. Here are a [couple of examples](https://plotly.com/examples/), and one for [Amazon](https://challenge-amazon.onrender.com/puchase_overview).

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define enhanced dashboard function
df['date'] = pd.to_datetime(df['date'])

def draw_state_dashboard(state):
    subset = df[df['ship_state'] == state.upper()]
    if subset.empty:
        print("No data for selected state.")
        return

    # Set up 2x2 subplot layout
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))

    # Pane 1: Bar plot of top categories
    top_cats = (subset.groupby('category')['qty']
                .sum()
                .sort_values(ascending=False)
                .head(6)
                .reset_index())
    sns.barplot(data=top_cats, x='category', y='qty', ax=axs[0, 0], palette='Blues_d', hue='category')
    axs[0, 0].set_title("Top Product Categories")
    axs[0, 0].set_ylabel("Quantity Sold")
    axs[0, 0].set_xlabel("")
    axs[0, 0].tick_params(axis='x', rotation=45)

    # Pane 2: Monthly revenue trend
    monthly = (subset.groupby(subset['date'].dt.to_period('M'))['amount_eu']
               .sum()
               .sort_index())
    monthly.index = monthly.index.to_timestamp()
    axs[0, 1].plot(monthly.index, monthly.values, marker='o')
    axs[0, 1].set_title("Monthly Revenue Trend")
    axs[0, 1].set_ylabel("Revenue")
    axs[0, 1].set_xlabel("Month")
    axs[0, 1].tick_params(axis='x', rotation=45)
    
    # Pane 3: Order status breakdown as a bar plot
    status_counts = (subset['status']
                     .value_counts()
                     .head(6)
                     .sort_values(ascending=False)
                     .reset_index())
    
    sns.barplot(data=status_counts, y='status', x='count', ax=axs[1, 0], palette='pastel', hue='status', orient='h')
    
    axs[1, 0].set_title("Top 5 Order Statuses")
    axs[1, 0].set_ylabel("Order Count")
    axs[1, 0].set_xlabel("")
    axs[1, 0].tick_params(axis='x', rotation=45)


    # Pane 4: Show summary stats as table
    total_orders = len(subset)
    total_revenue = subset['amount_eu'].sum()
    avg_order_value = subset['amount_eu'].mean()
    b2b_share = subset['B2B'].mean() * 100
    top_category = top_cats.iloc[0]['category']
    summary_data = pd.DataFrame({
        'Metric': ['Total Orders', 'Total Revenue', 'Average Order Value', 'B2B Share (%)', 'Top Category'],
        'Value': [total_orders, f"€{total_revenue:,.0f}", f"€{avg_order_value:,.0f}", f"{b2b_share:.1f}%", top_category]
    })
    axs[1, 1].axis('off')
    table = axs[1, 1].table(cellText=summary_data.values,
                           colLabels=summary_data.columns,
                           cellLoc='left', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)

    plt.tight_layout()
    plt.show()

# UI Dropdown
output = widgets.Output()
state_dropdown = widgets.Dropdown(
    options=sorted(df['ship_state'].dropna().unique()),
    description='Select a State:',
    style={'description_width': 'initial'}
)

def on_state_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output(wait=True)
            draw_state_dashboard(change['new'])

state_dropdown.observe(on_state_change)
display(state_dropdown, output)

Dropdown(description='Select a State:', options=('ANDAMAN & NICOBAR', 'ANDHRA PRADESH', 'AR', 'ARUNACHAL PRADE…

Output()

### Frequent Itemset Mining (Unordered Market Basket Analysis)

In market basket analysis, the goal is to find groups of products that are frequently bought together. There are two broad approaches:

- **Frequent Itemset Mining**: We don’t care about the order of products. We simply want to know which items often appear together in the same cart.
- **Association Rule Mining**: We care about order — e.g., if a customer buys A and B, they’re likely to also buy C. This introduces direction and confidence.

In this exercise, we focus on **frequent itemset mining**, i.e., **unordered** co-occurrence.

#### Key Concepts

We identify itemsets like: {'A', 'B', 'C'} that appear together in a minimum number of transactions. We will also be logging the support: the percentage of all orders that contain the entire itemset.

For example: ('J0389-TP-L', 'J0418-TP-L') with support = 0.025 means:   "These two products were bought together in _2.5% of all orders_".

There is no directionality here. We're not saying buying A leads to B, only that A and B often appear together. This is useful for:

- Identifying bundles
- Recommending related products
- Planning promotions or display layouts

In [14]:
! pip install pyfpgrowth

In [37]:
import pyfpgrowth

ANALYSIS_COLUMN = 'category' # Try base-SKU for online/category which would make more sense for offline settings

# Normalize SKUs by removing common size/variant suffixes
order_lists = df.groupby('order_id')[ANALYSIS_COLUMN].unique()

# Apply market basket analysis
patterns = pyfpgrowth.find_frequent_patterns(order_lists, 3)

# Filter itemsets with more than 1 item
unordered_sets = {k: v for k, v in patterns.items() if len(k) > 1}

df_patterns = pd.DataFrame(unordered_sets.items(), columns=['Itemset', 'Support'])
df_patterns['Size'] = df_patterns['Itemset'].apply(lambda x: len(x))
df_patterns.sort_values('Support', ascending=False)

,Itemset,Support,Size
21,"(Set, kurta)",908,2
20,"(Western Dress, kurta)",263,2
18,"(Set, Western Dress)",163,2
17,"(Top, kurta)",97,2
15,"(Set, Top)",86,2
11,"(Ethnic Dress, Set)",57,2
10,"(Ethnic Dress, Western Dress)",57,2
19,"(Set, Western Dress, kurta)",24,3
9,"(Ethnic Dress, Set, kurta)",12,3
16,"(Set, Top, kurta)",9,3
